In [1]:
import re
import datetime
import math
import psycopg2
import sshtunnel 
import pandas as pd
import numpy as np
from datetime import date, time,datetime, timedelta
from dateutil.rrule import rrule, DAILY

In [2]:
def execute_querie_test(c,sql):
        c.execute(sql)
        #db_list = c.fetchmany()
        field_names = [i[0] for i in c.description]
        query_results = pd.DataFrame(c.fetchall(),columns=field_names)
        
        return query_results
        #print(db_list)

In [3]:
def ssh_connect(sql):
    try:
        sshtunnel.SSH_TIMEOUT = 10.0

        with sshtunnel.SSHTunnelForwarder(
                ('3.83.88.154', 22),
                ssh_username='ec2-user',
                #CHANGE PATH TO rds-example.cer
                ssh_pkey='C:/Users/Maren/Documents/rds-example.cer',
                remote_bind_address=('stib-mivb.cjo3cnpt9pci.us-east-1.rds.amazonaws.com', 5432)
        ) as tunnel:
            conn = psycopg2.connect(
                    host='127.0.0.1',
                    port=tunnel.local_bind_port,     
                    database="stibmivbdb",
                    user="postgres",
                    password="stib-mivb-db1"
            )

            c = conn.cursor()

            #EXECUTE QUERY IN DB TO TEST
            query_results = execute_querie_test(c,sql)

            c.close()
            return query_results
    except Exception as e:
        print(e)

In [4]:
def getShortName(route_id,route_type):
    sql = "Select * from routes where route_id =" + str(route_id) + \
            " and route_type ='" + str(route_type) + \
            "' and date_format = '2022-09-03'"
    
    df_route = ssh_connect(sql)
    route_short = df_route['route_short_name'][0]
    
    return route_short

In [5]:
def getSchedule(route_id, route_type,date_from, date_to):
    sql = "Select * from schedules" + \
    " where route_id = " + str(route_id) + \
    " and route_type = " + str(route_type) + \
    " and start_date >= ('" + str(date_from)+ "'::date)" + \
    " and end_date <= ('" + str(date_to) + "'::date)" + \
    " order by stop_id,(start_date::date),arrival_time"
    
    print(sql)
    df_schedule = ssh_connect(sql)
    return df_schedule

In [6]:
def getValidDays(row):
    valid_days=[]
    
    if row['monday'] == 1:
        valid_days.append('Monday')
    if row['tuesday'] == 1:
        valid_days.append('Tuesday')
    if row['wednesday'] == 1:
        valid_days.append('Wednesday')
    if row['thursday'] == 1:
        valid_days.append('Thursday')
    if row['friday'] == 1:
        valid_days.append('Friday')
    if row['saturday'] == 1:
        valid_days.append('Saturday')
    if row['sunday'] == 1:
        valid_days.append('Sunday')
        
    return valid_days

In [7]:
#combine a date object and a time (as string) to datetime object
def getTime(date,time):
    hour = time[0:2]
    
    #check for times with hours > 23
    if int(hour) > 23:
        time_str = time
        new_time = time_str[0:2].replace(hour,str(0) + str(int(hour)-24)) + time_str[2:]
        c_date = pd.Timestamp.combine(date + timedelta(days=1),datetime.strptime(new_time, '%H:%M:%S').time())
    else:
        c_date = pd.Timestamp.combine(date,datetime.strptime(time, '%H:%M:%S').time())
    
    
    return c_date

In [8]:
def getCodeLigne(route_id,route_type):
    length = len(str(route_id))
    
    if length == 1:
        code_ligne = "00"+str(route_id)
    elif length == 2:
        code_ligne = "0"+str(route_id)
    elif length == 3:
        code_ligne = str(route_id)
        
    if route_type == 0:
        code_ligne = code_ligne + "t"
    elif route_type == 1:
        code_ligne = code_ligne + "m"
    elif route_type == 3:
        code_ligne = code_ligne + "b"
    
    return code_ligne

In [9]:
def getDirection(direction_id,route_id,route_type,headsigns):
    
    code_ligne = getCodeLigne(route_id,route_type)
    headsigns_str = (str(headsigns).replace('[','(')).replace(']',')')
    
    sql = "Select " + str(direction_id)+ " as direction_id, code_ligne,variante,succession,stop_id,descr_fr " +\
    "from actu_stops " +\
    "where code_ligne = '" + code_ligne +"' and descr_fr in " + headsigns_str +\
    " and terminus in " + headsigns_str
    
    print(sql)
    
    df_direction_id = ssh_connect(sql)
    
    return df_direction_id

In [10]:
#get the entries of vehicleposition table for a certain route, stop and timeframe
def getVehiclePositions(route_short,start_time_stamp, end_time_stamp):
    sql = """Select * from vehiclepositioncomplete 
        where lineid = """ + str(route_short) + \
        " and time >= " + str(start_time_stamp) + \
        " and time <= " + str(end_time_stamp) + \
        "order by pointid,time"

    df_position = ssh_connect(sql)
    df_position = df_position.replace(np.nan,0)
    return df_position

In [11]:
#select the closest time from the vehicle_postions (returns the index of the lower and the higher neighbour)
def find_neighbours(value, df, colname):
    
    #check if there is a bigger value then value in the dataframe
    if df[colname].iat[-1] >= value:
        if df[colname].iat[0] <= value:
        
            #check if there is exact match
            exactmatch = df[df[colname] == value]
            if not exactmatch.empty:
                return exactmatch.index
            else:
                #get closest neighbours
                lowerneighbour_ind = df[df[colname] < value][colname].idxmax()
                upperneighbour_ind = df[df[colname] > value][colname].idxmin()
                return [lowerneighbour_ind, upperneighbour_ind]
        else:
            return df.index[0]
    else:
        return df.index[-1]

In [12]:
def getSpeed(route_type):
    #tram 
    if route_type == 0:
        speed = 16.3
    #metro
    elif route_type == 1:
        speed = 27.9
    #bus
    elif route_type == 3:
        speed = 15.6
        
    return speed

In [13]:
#Get average distance from point for a stop excluding dinstances 0 and 1 (heuristic for vehicles with distance from point 1)
def getAverageDistance(df_vpos):
    average = df_vpos[(df_vpos.distancefrompoint != 0)&(df_vpos.distancefrompoint != 1)]['distancefrompoint'].mean()
    
    #only 0 and 1 as distance in df
    if math.isnan(average):
        return 1
    else:
        return average

In [14]:
#Estimate time it will take the vehicle to arrive at the point
def getTimeFromPoint(distance,df_vpos,speed):
    
    #Old vehicle with no exact distance from point take average distance from point for that stop
    if distance == 1:
        distance = getAverageDistance(df_vpos)
    
    if distance != 0 and distance != 1:
        #3.6 for km/h to m/s 1000 to get milliseconds
        mil_from_point = distance /(speed/3.6)*1000
        
        return mil_from_point
    
    #no prediciton possible vehicle just no there
    elif distance == 1:
        return 1
    
    else:
        return 0

In [15]:
#estimate the delay a vehicle will have
def getDelay(scheduled_stamp,v_pos_stamp, mil_from_point):
     #less than 2 minutes diff vehicle is punctual
    
    if mil_from_point != 1:
        delay = v_pos_stamp - mil_from_point - scheduled_stamp

        if delay < 120000:
            delay = 0
    
    else:
        #vehicle is late but no estimate possible 
        delay = 1
        
    return delay

In [16]:
def removeLetters(stop_id):
    if re.search('[a-zA-Z]', stop_id):
        stop_id = stop_id[:-1]

    return stop_id

In [17]:
def getAllLines(route_type):

    sql = "Select route_id,route_short_name,route_type from routes" +\
            " where route_type = '"+ str(route_type) + \
            "' and routes.date_format = ('2022-09-03'::date)"
    
    df_lines = ssh_connect(sql)
    
    return df_lines

In [18]:
def getPunctualityTimes(line_id, route_type):
    
    sql = "Select * from assessment_methods where method = 'punctuality' and" +\
        " route_id = "+ str(line_id) +"and date_to > ('2021-09-05' ::date) and route_type = '"+\
        str(route_type)+"' "+\
        "order by direction_id,assessment_methods.\"stopId\", (date_from::date),time_from"
    
    df_punct = ssh_connect(sql)
    
    return df_punct
    

In [40]:
#set the route type here (because running for all routy types at ones takes to much time)
route_type = 0
df_lines = getAllLines(route_type)
speed = getSpeed(route_type)
result = []

for ind, route in df_lines.iterrows():
    route_id = route['route_id']
    route_short = route['route_short_name']
    
    print("Starting_with_line: " + str(route_id))
    
    df_punct = getPunctualityTimes(route_id, route_type)
    
    #check if there is data for the line(new)
    if not df_punct.empty:
    #get schedule
        print(row)
        date_from = df_punct.min(axis=0)['date_from']
        date_to = df_punct.max(axis=0)['date_to']
        df_schedule = getSchedule(route_id, route_type ,date_from, date_to)

        #get Vehicle Positions
        start_time = getTime(date_from,df_punct.min(axis=0)['time_from'])
        end_time = getTime(date_to,df_punct.max(axis=0)['time_from'])
        start_time_stamp = (start_time - timedelta(minutes=150)).timestamp()*1e3
        end_time_stamp = (end_time - timedelta(minutes=90)).timestamp()*1e3
        df_vposition = getVehiclePositions(route_short,start_time_stamp, end_time_stamp)

        #Hier die ganzen Berechnungen
        #hier noch schleife für direction id einbauen
        for direction_id in range(0,2):

            stops = df_punct[df_punct.direction_id == direction_id]['stopId'].unique()
            for stop in stops:
                    #Filter df_punct for that stop
                    print(stop)
                    df_punct_stop = df_punct[(df_punct.stopId == stop)]
                    df_punct_stop = df_punct_stop.reset_index()

                    for index, row in df_punct_stop.iterrows():

                        #direction_id = row['direction_id']
                        #direction_id = 0
                        date_from = row['date_from']
                        date_to = row['date_to']
                        time_from = row['time_from']
                        time_to = row['time_to']

                        #print(date_from)
                        #print(date_to)

                        df_schedule_stop = df_schedule[(df_schedule.stop_id == stop)&(df_schedule.direction_id == direction_id)\
                                                    &(df_schedule.arrival_time >= time_from)&(df_schedule.arrival_time <= time_to)\
                                                   &(df_schedule.start_date == row['date_from'])&(df_schedule.end_date == row['date_to'])]

                        #get days for which schedule is valid
                        valid_days = getValidDays(row)

                        #loop over all days of the schedule
                        daterange = pd.date_range(date_from, date_to)

                        for c_date in daterange:

                            if (c_date.strftime("%A") in valid_days):

                                #get start and end time 
                                start_time = getTime(c_date,row['time_from'])
                                end_time = getTime(c_date,row['time_to'])

                                #check if data is present for that time:
                                if (start_time - timedelta(minutes=120)).timestamp()*1e3 >= 1630914886924:
                                    #strip letters from stop_id
                                    int_stop = removeLetters(stop)


                                    #Extend both by 30 minutes make sure to get also earlier or delayed vehicles (but also reduce it by two hours because timezone)
                                    start_time_stamp = (start_time - timedelta(minutes=150)).timestamp()*1e3
                                    end_time_stamp = (end_time - timedelta(minutes=90)).timestamp()*1e3

                                    df_vpos_stop = df_vposition[(df_vposition.time >= start_time_stamp)&(df_vposition.time <= end_time_stamp)\
                                                    &(df_vposition.pointid == int(int_stop))]

                                    df_vpos_stop = df_vpos_stop.reset_index()

                                    #check if vehicle position dataframe is empty (if it is no vehicle drove to the stop in that timeframe)

                                    #loop for the schedule
                                    for ind,scheduled in df_schedule_stop.iterrows():
                                        #create datetime object of arrival time

                                        arrival_time = getTime(c_date,scheduled['arrival_time'])
                                        arrival_time_stamp = (arrival_time - timedelta(minutes=120)).timestamp()*1e3

                                        try:
                                            if not df_vpos_stop.empty:
                                                #get closest neighbours for a schedule entry
                                                closest_pos = find_neighbours(arrival_time_stamp, df_vpos_stop, 'time')

                                                no_last_v = False
                                                #check if one or two neighbors were found
                                                if type(closest_pos) is list: 

                                                    lower_boundry = df_vpos_stop['time'][closest_pos[0]]
                                                    #for earlier only consider vehicles that were at max 90 seconds before scheduled time
                                                    if(abs(lower_boundry-arrival_time_stamp) <= abs(df_vpos_stop['time'][closest_pos[1]]-arrival_time_stamp)) \
                                                    and(abs(lower_boundry-arrival_time_stamp) + ((df_vpos_stop['distancefrompoint'][closest_pos[0]]/(speed/3.6))*1000) <= 90000):

                                                        index = closest_pos[0]
                                                    else:
                                                        index = closest_pos[1]
                                                        distance = df_vpos_stop['distancefrompoint'][index]
                                                        #diff = df_vpos_stop['time'][index] - arrival_time_stamp
                                                        mil_from_point = getTimeFromPoint(distance,df_vpos_stop,speed)

                                                        #check if after estimating position/ estimated arrival time is not to early
                                                        while (df_vpos_stop['time'][index] - mil_from_point) < (arrival_time_stamp - 90000):
                                                            if index < len(df_vpos_stop) - 1:
                                                                index = index + 1
                                                                distance = df_vpos_stop['distancefrompoint'][index]
                                                                #diff = df_vpos_stop['time'][index] - arrival_time_stamp
                                                                mil_from_point = getTimeFromPoint(distance,df_vpos_stop,speed)
                                                            else:
                                                                break
                                                                no_last_v = True

                                                else:
                                                    index = closest_pos

                                                    #check if the last vehicle did not come
                                                    if(df_vpos_stop['time'][index] <= arrival_time_stamp) \
                                                    and (abs(df_vpos_stop['time'][index]-arrival_time_stamp) + ((df_vpos_stop['distancefrompoint'][index]/(speed/3.6))*1000) > 120000):
                                                       no_last_v = True


                                                if no_last_v == False:
                                                    #time difference (in ms) between closest entry of vehicle positions and scheduled time
                                                    #diff = df_vpos_stop['time'][index] - arrival_time_stamp

                                                    #estimate time till point
                                                    closest_stamp = df_vpos_stop['time'][index]
                                                    distance = df_vpos_stop['distancefrompoint'][index]
                                                    mil_from_point = getTimeFromPoint(distance,df_vpos_stop,speed)
                                                    #delay = getDelay(mil_from_point)
                                                    delay = getDelay(arrival_time_stamp,closest_stamp, mil_from_point)

                                                else:
                                                    #delay -2 = last vehicle dont came/came early
                                                    closest_stamp = df_vpos_stop['time'][index]
                                                    #distance = df_vpos_stop['distancefrompoint'][index]
                                                    mil_from_point = 0 
                                                    delay = -2

                                            # delay -1 = no vehicle for that time and stop in vehicle positions
                                            else:
                                                    closest_stamp = 0
                                                    mil_from_point = 0
                                                    distance = 0
                                                    delay = -1
                                        except: 
                                            closest_stamp = 0
                                            mil_from_point = 0
                                            distance = 0
                                            delay = -1


                                        #Prepare result
                                        result.append(
                                        {
                                            'route_short_name': route_short,
                                            'route_id': route_id,
                                            'route_type': route_type,
                                            'direction_id': direction_id,
                                            'stop_id': stop,
                                            'date': c_date,
                                            'time': scheduled['arrival_time'],
                                            'timestamp':int(arrival_time_stamp),
                                            'estimated_arrival_stamp': int(closest_stamp - mil_from_point), 
                                            'delay': int(delay),
                                            'closest_stamp_vpos': closest_stamp,
                                            'distance_from_point': distance
                                        }
                                        )

        df_result = pd.DataFrame(result)
        df_result.to_csv(r'D:\OneDrive - Université Libre de Bruxelles\ULB\Data Mining\Project\Punctuality\puctuality_'+ str(route_id)+'_tram.csv',index=False)
        df_result = df_result[0:0]
        result = []
            

Starting_with_line: 5
index                   341
id                    99907
route_id                 25
direction_id              1
stopId                 6204
time_from          19:55:39
time_to            24:53:39
date_from        2021-09-19
date_to          2021-09-19
monday                    0
tuesday                   0
wednesday                 0
thursday                  0
friday                    0
saturday                  1
sunday                    0
method          punctuality
route_type                0
Name: 5, dtype: object
Select * from schedules where route_id = 5 and route_type = 0 and start_date >= ('2021-09-06'::date) and end_date <= ('2021-09-19'::date) order by stop_id,(start_date::date),arrival_time
0501
0511
0531
0601
0611
0621
0702
0703
0704
0705
0706
2318G
2338F
2543F
5221F
5700G
5701
5704F
5705
5706
5707
5714
6178
6253
6422F
6423F
9959F
0506
0516
0526
0606
0616
0626
0722
0723
0724
0725
0726
2200F
2337F
2537F
5219F
5770F
5771
5772
5773
5776
5806
6177
6179


0089
1042
5281G
5501
5502
5503
5504
5507
5508
5601
5602
5603
5604
5605
9551
9552
9553
9556
0089
5272F
5273F
5531
5555
5556
5559
5560
5561F
5562
5655
5656
5657
5658
5659
9557
9575
9577
9578
Starting_with_line: 15
index                   184
id                   100535
route_id                 40
direction_id              1
stopId                 9578
time_from          19:42:59
time_to            24:51:59
date_from        2021-09-19
date_to          2021-09-19
monday                    0
tuesday                   0
wednesday                 0
thursday                  0
friday                    0
saturday                  1
sunday                    0
method          punctuality
route_type                0
Name: 4, dtype: object
Select * from schedules where route_id = 15 and route_type = 0 and start_date >= ('2021-09-01'::date) and end_date <= ('2021-09-19'::date) order by stop_id,(start_date::date),arrival_time
0702
0703
0704
0705
0706
1110
1290
1926G
1984G
2165F
2851F
2869
5006
5007

1985G
2334F
2336G
2404F
2603
2604F
2608F
2764G
2765
5016F
5018F
5115F
5116
5119
5120
5121F
5123
5719H
5720F
5721G
5722F
5723G
5916F
5917F
5921G
6078
6162
6314
6428F
9972F
1258G
1984G
2333F
2462F
2463F
2663G
2665F
2667F
2668
2669F
2708G
2709F
2710G
5066F
5068F
5151
5152G
5155
5158
5159F
5161F
5175
5756F
5920H
5952F
5953F
6077F
6109
6361
6427F
6430F


In [38]:
df_punct.min(axis=0)['time_from']

nan

In [39]:
getPunctualityTimes(76, 0)

,id,route_id,direction_id,stopId,time_from,time_to,date_from,date_to,monday,tuesday,wednesday,thursday,friday,saturday,sunday,method,route_type
